In [1]:
import requests
import requests.auth
import pandas as pd
from psaw import PushshiftAPI
api = PushshiftAPI()
import datetime as dt
from textblob import TextBlob

### Pushshift API wrapper query
    * automatic rate limiting

In [3]:
# get all threads from the bitcoinmarket subreddit, gen is a generator 
start_epoch=int(dt.datetime(2013, 1, 1).timestamp())
gen = api.search_submissions(after=start_epoch,
        subreddit='bitcoinmarkets', #specify the subforum
        sort = 'asc',
        # filter=['url','author', 'title', 'subreddit'], # choose which dictionary keys to display
        limit=100000)

In [4]:
# iterate through the generator to create a submission array
# x.d_ is a method that converts each submission object to a dictionary
results = list(map(lambda x: x.d_, gen))

In [5]:
df = pd.DataFrame(results)
df.to_csv('bitcionmarkets_threads.csv')
df.to_json('')